# Рекомендация тарифов

## Откройте и изучите файл

In [8]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import dump
from sklearn.linear_model import LogisticRegression 
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [10]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [11]:
df.duplicated().sum()

0

Вывод: все библиотеки загрузили, файл открыли. Изучили: с типами данных все в порядке, пропусков нет, дубликатов нет. Можно приступать к следующему шагу проекта - разбивка данных на выборки.

## Разбейте данные на выборки

Разобьем данные на выборки: Обучающую, валидационную, тестовую в соотношении 60-20-20 соотвественно.

In [12]:

features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']
# делим выборку на train, valid на 0,6 и 0,4 соответственно
features_train, features_valid, target_train, target_valid= train_test_split(features,target, test_size=0.4, random_state=12345,stratify=target)
# делим выборку valid на valid и test 0,4 делим пополам
features_valid, features_test, target_valid, target_test = train_test_split(features_valid,target_valid, test_size=0.5, random_state=12345,stratify=target_valid)


print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)

(1928, 4) (643, 4) (643, 4)
(1928,) (643,) (643,)


## Исследуйте модели

Исследуем модели:
1 Дерево
2 Случайный лес
3 Логическая регрессия

Исследуем модель Дерево

In [13]:
best_model_tree = model
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) #  создадим модель, указав max_depth=depth 
    model.fit(features_train,target_train) #  обучим модель 
    predictions_valid = model.predict(features_valid) #  найдем предсказания на валидационной выборке 

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))


max_depth = 1 : 0.7402799377916018
max_depth = 2 : 0.7729393468118196
max_depth = 3 : 0.7776049766718507
max_depth = 4 : 0.7542768273716952
max_depth = 5 : 0.7853810264385692


Лучшее значение на глубине 5
Попробуем взять больше ветвей


In [14]:
for depth in range(1, 15):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) #  создадим модель, указав max_depth=depth 
    model.fit(features_train,target_train) #  обучим модель 
    predictions_valid = model.predict(features_valid) #  найдем предсказания на валидационной выборке 

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7402799377916018
max_depth = 2 : 0.7729393468118196
max_depth = 3 : 0.7776049766718507
max_depth = 4 : 0.7542768273716952
max_depth = 5 : 0.7853810264385692
max_depth = 6 : 0.7744945567651633
max_depth = 7 : 0.7869362363919129
max_depth = 8 : 0.80248833592535
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7729393468118196
max_depth = 11 : 0.7620528771384136
max_depth = 12 : 0.7527216174183515
max_depth = 13 : 0.744945567651633
max_depth = 14 : 0.7558320373250389


Лучшее значение на глубине 8.
Остановимся на ней

Теперь исследуем модель случайный лес.

In [15]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # создадим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучим модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model# сохраним наилучшую модель
        best_result = result #  сохраним наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7869362363919129


In [16]:
best_model = None
best_result = 0
for est in range(11, 30):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # создадим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result:
        best_model = model# сохраните наилучшую модель
        best_result = result #  сохраните наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625


In [17]:
best_model_forest = None
best_result = 0
for est in range(40, 50):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # создадим модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result:
        best_model_forest = model
 # сохраните наилучшую модель
        best_result = result #  сохраните наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Accuracy наилучшей модели на валидационной выборке: 0.8055987558320373


Лучший результат лежит в диапазоне от 40-50. 


Теперь исследуем модель Логическая регрессия

In [19]:
model = LogisticRegression(random_state=12345, solver = 'lbfgs', max_iter = 100)# создадим модель с заданными параметрами, возьмем количество итераций по умолчанию 
model.fit(features_train, target_train)
model_predict = model.predict(features_valid) 
model.score(features, target) 


0.7476664592408214

Вывод: в этом разделе мы исследовали качество разных моделей. Самое низкое значение accurasy =0.8055987558320373 у нас плучитось на модели Логичкская регрессия.Самое высокое на модели Случайный лес = 0.7947122861586314.

## Проверьте модель на тестовой выборке

Теперь проверим модель на тестовой выборке

In [20]:

test_predictions = best_model_forest.predict(features_test)
accuracy_score(target_test, test_predictions) 

print("Тестовая выборка:", accuracy_score(target_test, test_predictions))


Тестовая выборка: 0.7993779160186625


Значение accurasy на тестовой модели немного уменьшилось. 

## (бонус) Проверьте модели на адекватность

In [21]:
import numpy as np 
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier ( strategy = "most_frequent" )
dummy_clf . fit (features_train, target_train)
DummyClassifier(strategy='most_frequent') 
dummy_clf . predict (features)
dummy_clf . score (features_test, target_test)

0.6936236391912908